<a href="https://colab.research.google.com/github/yurityger/ma_sales/blob/python/ab_port.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Проаналізуємо результати A/B-тестування за допомогою статистичних методів в Python та створимо візуалізацію, що демонструє ключові конверсійні метрики

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [ ]:
# підключаємо результат SQL запита з Google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Mate_Courses/Pandas/Panda_files_ma

/content/drive/MyDrive/Mate_Courses/Pandas/Panda_files_ma


In [ ]:
ab = pd.read_csv("sql_for_ab_test.csv")
ab.head(20)

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1
5,2020-11-02,Jordan,desktop,Asia,Organic Search,2,2,new account,1
6,2020-11-02,Venezuela,mobile,Americas,Organic Search,2,2,new account,1
7,2020-11-02,North Macedonia,desktop,Europe,Organic Search,2,2,new account,1
8,2020-11-02,Georgia,desktop,Asia,Organic Search,2,2,new account,1
9,2020-11-02,Tunisia,desktop,Africa,Organic Search,2,1,new account,1


In [ ]:
# створимо сумарну таблицю з результатами кожного тесту необхідним метрик
# Збираємо метрики
metrics = {
    "add_payment_info/session": "add_payment_info",
    "add_shipping_info/session": "add_shipping_info",
    "begin_checkout/session": "begin_checkout",
    "new_accounts/session": "new account",
    "session_with_orders/session": "session with orders",
}

total_results = []

# Рахуємо сесії для кожної групи
for metric_name, event_name in metrics.items():
    for test_group, group_data in ab.groupby("test"):
        group1 = group_data[group_data["test_group"] == 1]
        group2 = group_data[group_data["test_group"] == 2]

        # Підраховуємо суму "value" для кожної групи та події
        value_sum1 = group1[group1["event_name"] == event_name]["value"].sum()
        value_sum2 = group2[group2["event_name"] == event_name]["value"].sum()

        # Підраховуємо кількість сесій (для події "session")
        sessions1 = group1[group1["event_name"] == "session"]["value"].sum()
        sessions2 = group2[group2["event_name"] == "session"]["value"].sum()

        # Формуємо таблицю спостережуваних частот
        observed = np.array([[value_sum1, sessions1 - value_sum1], [value_sum2, sessions2 - value_sum2]])

        # Підключаємо тест статистично значущих результатів
        chi2, p_value, _, _ = chi2_contingency(observed, correction=False)

        # Задаємо рівень допуску
        is_significant = p_value < 0.05

        # Рахуємо конверсії
        conversion1 = value_sum1 / sessions1 if sessions1 > 0 else 0
        conversion2 = value_sum2 / sessions2 if sessions2 > 0 else 0

        # Визначаємо різницю у відсотках
        conversion_diff = (conversion2 - conversion1) / conversion1 * 100

        z_stat = np.sqrt(chi2) * np.sign(conversion_diff)

        # Додаємо результати до списку
        total_results.append([
            test_group, metric_name, value_sum1, sessions1, conversion1,
            value_sum2, sessions2, conversion2, conversion_diff, z_stat, p_value, is_significant
        ])

# Формуємо DataFrame з результатами
total_results = pd.DataFrame(total_results, columns=[
    "test", "metric", "value_sum_gr1", "sessions_gr1", "conversion_gr1",
    "value_sum_gr2", "sessions_gr2", "conversion_gr2", "conversion_diff",
    "z_stat", "p_value", "is_significant"
])

total_results.head()

,test,metric,value_sum_gr1,sessions_gr1,conversion_gr1,value_sum_gr2,sessions_gr2,conversion_gr2,conversion_diff,z_stat,p_value,is_significant
0,1,add_payment_info/session,1988,45362,0.043825,2229,45193,0.049322,12.542021,3.924884,0.000087,True
1,2,add_payment_info/session,2344,50637,0.046290,2409,50244,0.047946,3.576911,1.240994,0.214608,False
2,3,add_payment_info/session,3623,70047,0.051722,3697,70439,0.052485,1.474630,0.643172,0.520112,False
3,4,add_payment_info/session,3731,105079,0.035507,3601,105141,0.034249,-3.541234,-1.571106,0.116158,False
4,1,add_shipping_info/session,3034,45362,0.066884,3221,45193,0.071272,6.560481,2.603571,0.009226,True


In [ ]:
# Збрегаємо необхідну нам таблицю
total_results.to_csv('total_for_tablue_portfolio_1.csv', index=False)

In [ ]:
# Розрахуємо в розрізі девайсів
# Збираємо метрики
metrics = {
    "add_payment_info/session": "add_payment_info",
    "add_shipping_info/session": "add_shipping_info",
    "begin_checkout/session": "begin_checkout",
    "new_accounts/session": "new account",
    "session_with_orders/session": "session with orders",
}

results = []

# Підрахунок сесій для кожної групи та девайса
for metric_name, event_name in metrics.items():
    for test_group, group_data in ab.groupby("test"):
        for device, device_data in group_data.groupby("device"):
            group1 = device_data[device_data["test_group"] == 1]
            group2 = device_data[device_data["test_group"] == 2]

            # Сумуємо значення "value" для кожної групи та події
            value_sum1 = group1[group1["event_name"] == event_name]["value"].sum()
            value_sum2 = group2[group2["event_name"] == event_name]["value"].sum()

            # Підраховуємо кількість сесій
            sessions1 = group1[group1["event_name"] == "session"]["value"].sum()
            sessions2 = group2[group2["event_name"] == "session"]["value"].sum()

            # Перевірка
            if sessions1 > 0 and sessions2 > 0:
                observed = np.array([
                    [value_sum1, sessions1 - value_sum1],
                    [value_sum2, sessions2 - value_sum2]
                ])

                # Підключаємо тест статистично значущих результатів
                chi2, p_value, _, _ = chi2_contingency(observed, correction=False)

                # Задаємо рівень допуску
                is_significant = p_value < 0.05

                # Рахуємо конверсії
                conversion1 = value_sum1 / sessions1 if sessions1 > 0 else 0
                conversion2 = value_sum2 / sessions2 if sessions2 > 0 else 0

                # Визначаємо різницю у відсотках
                conversion_diff = (conversion2 - conversion1) / conversion1 * 100

                z_stat = np.sqrt(chi2) * np.sign(conversion_diff)

                results.append([
                    test_group, metric_name, device, value_sum1, sessions1, conversion1, value_sum2, sessions2,
                    conversion2, conversion_diff, z_stat, p_value, is_significant
                ])
            else:
                # Якщо немає даних для сесій, додаємо NaN
                results.append([
                    test_group, metric_name, device, value_sum1, sessions1, value_sum2, sessions2,
                ])

# Створюємо DataFrame з результатами
results_df = pd.DataFrame(results, columns=[
    "test", "metric", "device", "value_sum_gr1", "sessions_gr1", "conversion_gr1",
    "value_sum_gr2", "sessions_gr2", "conversion_gr2", "conversion_diff",
    "z_stat", "p_value", "is_significant"
])

results_df.head()


,test,metric,device,value_sum_gr1,sessions_gr1,conversion_gr1,value_sum_gr2,sessions_gr2,conversion_gr2,conversion_diff,z_stat,p_value,is_significant
0,1,add_payment_info/session,desktop,1130,26467,0.042695,1256,26417,0.047545,11.360819,2.686998,0.007210,True
1,1,add_payment_info/session,mobile,810,17896,0.045262,942,17767,0.053020,17.140683,3.389330,0.000701,True
2,1,add_payment_info/session,tablet,48,999,0.048048,31,1009,0.030723,-36.056739,-1.996608,0.045868,True
3,2,add_payment_info/session,desktop,1314,29497,0.044547,1401,29380,0.047686,7.045601,1.815587,0.069434,False
4,2,add_payment_info/session,mobile,978,20017,0.048858,961,19756,0.048643,-0.440088,-0.099562,0.920692,False


In [ ]:
# Збрегаємо таблицю з девайсами
results_df.to_csv('device_for_tablue_portfolio_1.csv', index=False)


За результатами тестів можемо сказати, що суттєві показники росту показав лише первий тест для метрик:
* add_payment_info
* add_shipping_info
* begin_checkout
В розрізі девайсів суттємо позитивну динаміку показують *Desktop* та іноді *Mobile*. Результат *Tablet* майже завжди істотно негативний

Залишаю посилання:
- на візуалізацію:
[відкрити Tablue](https://public.tableau.com/views/ABPort/Dashboard1?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)
- на першу сумарну
[таблицю Total](https://drive.google.com/file/d/1oFnvZamecqxK1KCuzf_5GWNLZ23fnLVC/view?usp=sharing)
- на таблицю [в розрізі девайсів](https://drive.google.com/file/d/1-0pEQbwiftAdR3_lVne1r0smCXpwRdac/view?usp=sharing)






---

